## 1. checking if it loads correctly

In [1]:
from tqdm.auto import tqdm

In [2]:
from nerus.load import load_norm

In [3]:
records = load_norm('../../data/nerus/nerus_lenta.jsonl.gz')

In [4]:
type(records)

generator

In [5]:
r=next(records)
r.text, r.spans[:2]

('Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
 [Span(
      start=36,
      stop=52,
      type='PER'
  ), Span(
      start=82,
      stop=88,
      type='LOC'
  )])

## 2. loading, splitting into train/test and saving

In [6]:
from nerus.load import load_norm
def load_nerus(fn):
    entries = []
    for r in tqdm(load_norm(fn), total=739295):
        entries.append({'raw': r.text, 'entities': [tuple(x) for x in r.spans]})
    return entries
entries = load_nerus('../../data/nerus/nerus_lenta.jsonl.gz')

In [7]:
entries[0]

{'raw': 'Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
 'entities': [(36, 52, 'PER'),
  (82, 88, 'LOC'),
  (149, 160, 'ORG'),
  (172, 181, 'PER'),
  (251, 260, 'LOC'),
  (262, 270, 'LOC'),
  (272, 280, 'LOC'),
  (283, 301, 'LOC'),
  (313, 324, 'LOC'),
  (383, 389, 'LOC'),
  (560, 568, 'ORG')]}

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
ds_train, ds_test = train_test_split(entries, test_size=20000, random_state=42)
print(len(ds_train), len(ds_test))

719295 20000


In [10]:
import gzip, json
def save_entries(fn, entries): # '../data/datasets/nerus.jsonl.gz'
    with gzip.open(fn, 'wt') as f:
        for z in tqdm(entries):
            json.dump(z, f, ensure_ascii=False)
            f.write('\n')
        f.close()

In [11]:
save_entries('../../data/datasets/nerus_train.jsonl.gz', ds_train)

In [12]:
save_entries('../../data/datasets/nerus_test.jsonl.gz', ds_test)

## 3. Loading and checking everything went smoothly

In [13]:
import gzip, json
def load_entries(fn): # '../data/datasets/nerus.jsonl.gz'
    entries = []
    with gzip.open(fn, 'r') as f:
        for line in tqdm(f):
            entry = json.loads(line)
            entries.append(entry)
    return entries
    #del entries

In [14]:
ds_test = load_entries('../../data/datasets/nerus_test.jsonl.gz')

In [15]:
ds_train = load_entries('../../data/datasets/nerus_train.jsonl.gz')